In this project, we will analyze the result of an A/B test where the first gate in Cookie Cats game was moved from level 30 to level 40. In particular, we will analyze the impact on player retention and game rounds.

The data is from 90,189 players that installed the game while the AB-test was running. The variables are:

-userid: a unique number that identifies each player.

-version: whether the player was put in the control group (gate_30 - a gate at level 30) or the test group (gate_40 - a gate at level 40).

-sum_gamerounds: the number of game rounds played by the player during the first week after installation

-retention_1: did the player come back and play 1 day after installing?

-retention_7: did the player come back and play 7 days after installing?

In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [54]:
import scipy.stats as stats
from scipy.stats import shapiro

In [59]:
df = pd.read_csv("../input/mobile-games-ab-testing/cookie_cats.csv")

In [60]:
df.head(5)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


## STATISTICS Summary

In [61]:
# Number of Unique User
df.userid.nunique() == df.shape[0]

True

In [62]:
df.describe([0.01, 0.05, 0.1, 0.2, 0.8, 0.9, 0.95, 0.99])[["sum_gamerounds"]]

,sum_gamerounds
count,90189.000000
mean,51.872457
std,195.050858
min,0.000000
1%,0.000000
5%,1.000000
10%,1.000000
20%,3.000000
50%,16.000000
80%,67.000000


## A/B Groups & Target Summary Stats

In [63]:
df.groupby("version").sum_gamerounds.agg(["count", "median", "mean", "std", "max"])

,count,median,mean,std,max
version,,,,,
gate_30,44700,17,52.456264,256.716423,49854
gate_40,45489,16,51.298776,103.294416,2640


## Outliers

In [64]:
df = df[df.sum_gamerounds < df.sum_gamerounds.max()]

df.describe([0.01, 0.05, 0.1, 0.2, 0.8, 0.9, 0.95, 0.99])[["sum_gamerounds"]]

,sum_gamerounds
count,90188.000000
mean,51.320253
std,102.682719
min,0.000000
1%,0.000000
5%,1.000000
10%,1.000000
20%,3.000000
50%,16.000000
80%,67.000000


## Some Details

In [65]:
df.groupby("sum_gamerounds").userid.count().reset_index().head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
sum_gamerounds,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
userid,3994,5538,4606,3958,3629,2992,2861,2379,2267,2013,1752,1654,1570,1594,1519,1446,1342,1269,1228,1158


In [66]:
# How many users reached gate 30 & gate 40 levels?
df.groupby("sum_gamerounds").userid.count().loc[[30, 40]]

sum_gamerounds
30    642
40    505
Name: userid, dtype: int64

In [67]:
# A/B Groups & Target Summary Stats

df.groupby("version").sum_gamerounds.agg(["count", "median", "mean", "std", "max"])

,count,median,mean,std,max
version,,,,,
gate_30,44699,17,51.342111,102.057598,2961
gate_40,45489,16,51.298776,103.294416,2640


In [69]:
df.groupby(["version","retention_1"]).sum_gamerounds.agg(["count", "median", "mean", "std", "max"])

count  median       mean         std   max
version retention_1                                            
gate_30 False        24665       6  16.359092   36.528426  1072
        True         20034      48  94.411700  135.037697  2961
gate_40 False        25370       6  16.340402   35.925756  1241
        True         20119      49  95.381182  137.887256  2640

In [70]:
df.groupby(["version","retention_7"]).sum_gamerounds.agg(["count", "median", "mean", "std", "max"])

count  median        mean         std   max
version retention_7                                             
gate_30 False        36198      11   25.796508   43.316158   981
        True          8501     105  160.117516  179.358560  2961
gate_40 False        37210      11   25.856356   44.406112  2640
        True          8279     111  165.649837  183.792499  2294

In [71]:
df["NewRetention"] = list(map(lambda x, y: str(x)+ "-" +str(y), df.retention_1, df.retention_7))

df.groupby(["version", "NewRetention"]).sum_gamerounds.agg(["count", "median", "mean", "std", "max"]).reset_index()

,version,NewRetention,count,median,mean,std,max
0,gate_30,False-False,22840,6,11.819746,21.642643,981
1,gate_30,False-True,1825,43,73.169315,93.222330,1072
2,gate_30,True-False,13358,33,49.694490,58.125396,918
3,gate_30,True-True,6676,127,183.886309,189.626390,2961
4,gate_40,False-False,23597,6,11.913294,20.901020,547
5,gate_40,False-True,1773,47,75.261139,94.478048,1241
6,gate_40,True-False,13613,32,50.025490,60.924587,2640
7,gate_40,True-True,6506,133,190.282355,194.220077,2294


# A/B Testing

#### Define A/B groups

In [72]:
df.version = np.where(df.version == "gate_30", "A", "B")

df.head(10)

,userid,version,sum_gamerounds,retention_1,retention_7,NewRetention
0,116,A,3,False,False,False-False
1,337,A,38,True,False,True-False
2,377,B,165,True,False,True-False
3,483,B,1,False,False,False-False
4,488,B,179,True,True,True-True
5,540,B,187,True,True,True-True
6,1066,A,0,False,False,False-False
7,1444,B,2,False,False,False-False
8,1574,B,108,True,True,True-True
9,1587,B,153,True,False,True-False


In [73]:
df[df["version"] == "A"]

,userid,version,sum_gamerounds,retention_1,retention_7,NewRetention
0,116,A,3,False,False,False-False
1,337,A,38,True,False,True-False
6,1066,A,0,False,False,False-False
11,2101,A,0,False,False,False-False
13,2179,A,39,True,False,True-False
...,...,...,...,...,...,...
90179,9998576,A,14,True,False,True-False
90180,9998623,A,7,False,False,False-False
90182,9999178,A,21,True,False,True-False
90183,9999349,A,10,False,False,False-False


In [79]:
shapiro(df[df["version"] == "B"]["sum_gamerounds"])[1]

/opt/conda/lib/python3.7/site-packages/scipy/stats/morestats.py:1681: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


0.0

## A/B testing function

In [85]:
def AB_Test(dataframe, group, target):
    
    # split A/B
    groupA = dataframe[dataframe[group] == "A"][target]
    groupB = dataframe[dataframe[group] == "B"][target]
    
    # Assumption: Normality
    ntA = shapiro(groupA)[1] < 0.05
    ntB = shapiro(groupB)[1] < 0.05
    
    # H0: Distribution is Normal! - False
    # H1: Distribution is not Normal! - True
    
    if (ntA == False)&(ntB == False):
        leveneTest = stats.levene(groupA, groupB)[1] < 0.05
        if leveneTest == False:
            ttest = stats.ttest_ind(groupA, groupB, eqaul_var=True)[1]
        else:
            ttest = stats.ttest_ind(groupA, groupB, eqaul_var=False)[1]
    else:
        ttest = stats.mannwhitneyu(groupA, groupB)[1]
        
    # result
    temp = pd.DataFrame({
        "AB Hypothesis":[ttest < 0.05],
        "p-value":[ttest]
    })
    temp["Test Type"] = np.where((ntA == False) & (ntB == False), "Parametric", "Non-Parametric")
    temp["AB Hypothesis"] = np.where(temp["AB Hypothesis"] == False, "Fail to Reject H0", "Reject H0")
    temp["Comment"] = np.where(temp["AB Hypothesis"] == "Fail to Reject H0", "A?B groups are similar", "A/B groups are not similar!")
    
    
    # cols
    if (ntA == False)&(ntB == False):
        temp["Homo genereity"] = np.where(leveneTest == False, "Yes", "No")
        temp = temp[["Test TYpe", "Homogeneity", "AB Hypothesis", "p-value", "Comment"]]
    else:
        temp = temp[["Test Type", "AB Hypothesis", "p-value", "Comment"]]
        
    print("# A?B Testing Hypothesis")
    print("H0: A==B")
    print("H1: A!=B", "\n") 

    return temp
AB_Test(dataframe=df,group="version",target="sum_gamerounds")

# A?B Testing Hypothesis
H0: A==B
H1: A!=B 



/opt/conda/lib/python3.7/site-packages/scipy/stats/morestats.py:1681: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


,Test Type,AB Hypothesis,p-value,Comment
0,Non-Parametric,Reject H0,0.025446,A/B groups are not similar!


There are statistically significant difference between two groups about moving first gate from level 30 to level 40 for game rounds.

## Which level has more advantages in terms of player retention?

In [88]:
df.groupby("version").retention_1.mean()

version
A    0.448198
B    0.442283
Name: retention_1, dtype: float64

In [89]:
df.groupby("version").retention_7.mean()

version
A    0.190183
B    0.182000
Name: retention_7, dtype: float64